In [1]:
%matplotlib inline
import csv
import json
import os
import sys
import time

import cv2
import numpy as np

file = "data/s1/SSSM-B3-00090.MOV"

nFrames = 20 #检测的帧数量

name = str(file).split("/")[-1].split(".")[0]

direction_res = "results/s1/direction/"

corner_res = "results/s1/corner/"

hand_res = "results/s1/hand/"

hands = open(hand_res+name+".txt").read().split('\n')[:-1]

for i in range(len(hands)):
    hands[i] = hands[i].split()
    if int(hands[i][0]) < nFrames:
        hands[i][0] = int(hands[i][0])
        hands[i][3] = float(hands[i][3])
    else:
        break
hands = hands[:i]
hands

[[0, '1381', '760', 0.789409875869751],
 [1, '1384', '760', 0.8409484028816223],
 [2, '1384', '760', 0.8725989460945129],
 [3, '1384', '761', 0.8481162190437317],
 [4, '1384', '761', 0.308960884809494],
 [5, '1395', '762', 0.6441797018051147],
 [6, '1392', '762', 0.8909472227096558],
 [7, '1397', '761', 0.8635904788970947],
 [8, '1396', '761', 0.4396103620529175],
 [9, '1396', '762', 0.6291674375534058],
 [10, '1400', '761', 0.751805305480957],
 [11, '1404', '761', 0.8011921048164368],
 [12, '1404', '761', 0.669951319694519],
 [13, '1404', '761', 0.8825952410697937],
 [14, '1404', '761', 0.8388943076133728],
 [15, '1404', '761', 0.892366886138916],
 [16, '1404', '761', 0.6459530591964722],
 [17, '1404', '761', 0.8661409616470337],
 [18, '1404', '761', 0.8389847874641418],
 [19, '1404', '761', 0.8560546040534973]]

In [2]:
def takeKey(elem):
    return elem[0][3]

In [3]:
detection_results = []
i = 0
while i < nFrames:
    pic_name = name+"_"+str(i)

    with open(corner_res+pic_name+".txt",'r') as f:
        corner = f.readline()[1:-1].split()

    #判断是否检测到
    if len(corner)< 4:
        i = i + 1
        continue

    for j in range(len(hands)):
        if hands[j][0] == i:
            finger = hands[j]
            detection_results.append([finger,corner])#保存手部关键点和手机角

    i += 1

detection_results.sort(key = takeKey,reverse = True)
dresult = detection_results[0]
dresult

[[15, '1404', '761', 0.892366886138916], ['1432', '858', '1450', '876']]

In [ ]:
frame_no = int(dresult[0][0])

# Read video
video = cv2.VideoCapture(str(file))

# Exit if video not opened.
if not video.isOpened():
    print("Could not open video")
    sys.exit()

# Read first frame.
while frame_no >= 0:
    ok, frame = video.read()
    if not ok:
        print('Cannot read video file')
        sys.exit()
    frame_no -= 1

finger = dresult[0]
corner = dresult[1]

finger_box = tuple([int(finger[1]),int(finger[2]),20,20])
phone_box = tuple([int((int(corner[0])+int(corner[2]))/2-10),int((int(corner[1])+int(corner[3]))/2-10),20,20])

X = []
Y = []

np.array(X)
np.array(Y)

finger_center = (int(finger_box[0] + finger_box[2] / 2), int(finger_box[1] + finger_box[3] / 2))
phone_center = (int(phone_box[0] + phone_box[2] / 2), int(phone_box[1] + phone_box[3] / 2))
X.append(finger_center[0]-phone_center[0])
Y.append(finger_center[1]-phone_center[1])

# Set up tracker.
finger_tracker = cv2.TrackerCSRT_create()
phone_tracker = cv2.TrackerCSRT_create()

finger_ok = finger_tracker.init(frame, finger_box)
phone_ok = phone_tracker.init(frame, phone_box)
sys.exit()
while True:
        # Read a new frame
        ok, frame = video.read()
        if not ok:
            break

        # Start timer
        timer = cv2.getTickCount()

        # Update tracker
        finger_ok, finger_box = finger_tracker.update(frame)
        phone_ok, phone_box = phone_tracker.update(frame)

        # Calculate Frames per second (FPS)
        fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);
        break
        # Draw bounding box
        if finger_ok :
            # Tracking success
            p1 = (int(finger_box[0]), int(finger_box[1]))
            p2 = (int(finger_box[0] + finger_box[2]), int(finger_box[1] + finger_box[3]))
            p3 = (int(phone_box[0]), int(phone_box[1]))
            p4 = (int(phone_box[0] + phone_box[2]), int(phone_box[1] + phone_box[3]))
            cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
            cv2.rectangle(frame, p3, p4, (255,0,0), 2, 1)
            finger_center = (int(finger_box[0] + finger_box[2] / 2), int(finger_box[1] + finger_box[3] / 2))
            phone_center = (int(phone_box[0] + phone_box[2] / 2), int(phone_box[1] + phone_box[3] / 2))
            # X.append(finger_center[0])
            # Y.append(finger_center[1])
            X.append(finger_center[0]-phone_center[0])
            Y.append(finger_center[1]-phone_center[1])
        else:
            # Tracking failure
            cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)

        # Display tracker type on frame
        cv2.putText(frame, "CSRT" + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2);

        # Display FPS on frame
        cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2);

#         plt.imshow(frame)
#         plt.show()

        # Exit if ESC pressed
        k = cv2.waitKey(0) & 0xff
        if k == 32:
            label = 1
        else:
            label = 0

        l.append(label)


rows= []
direction = open(direction_res+name+".txt").readline()

if direction == '1':
    for i in range(len(X)):
        row = {"frame":i,"X":X[i]-X[0],"Y":Y[0]-Y[i]}
        rows.append(row)
elif direction == '2':
    for i in range(len(X)):
        row = {"frame":i,"X":Y[i]-Y[0],"Y":X[i]-X[0]}
        rows.append(row)
elif direction == '3':
    for i in range(len(X)):
        row = {"frame":i,"X":X[0]-X[i],"Y":Y[i]-Y[0]}
        rows.append(row)
elif direction == '4':
    for i in range(len(X)):
        row = {"frame":i,"X":Y[0]-Y[i],"Y":X[0]-X[i]}
        rows.append(row)

res_path = "results/"+name
if not os.path.exists(res_path):
    os.mkdir(res_path)


with open("results/"+name+"/raw_trajectory.csv","w") as f:
    rst_csv = csv.DictWriter(f,["frame","X","Y"])
    rst_csv.writeheader()
    rst_csv.writerows(rows)

video.release()
cv2.destroyAllWindows()